# Layer and Block

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 2.5246e-02,  1.4953e-01,  1.8365e-01, -7.6597e-02,  1.2013e-01,
         -2.1167e-01, -1.6896e-01, -1.4491e-01,  1.5909e-02,  2.2795e-01],
        [ 1.0796e-04,  1.0958e-01,  1.7935e-01, -7.2118e-02,  1.3327e-01,
         -2.1855e-01,  3.1318e-02, -8.4138e-02, -1.7168e-01,  1.9097e-01]],
       grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # forward propagation
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


net = MLP()
net(X)

tensor([[-0.0425, -0.1382, -0.0625,  0.0780, -0.0922,  0.0286,  0.0583,  0.0351,
         -0.0045,  0.2278],
        [-0.1072, -0.0880, -0.0173,  0.1719, -0.0332,  0.0374,  0.1106,  0.0293,
         -0.0917,  0.1882]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0811,  0.3077, -0.1833,  0.1942, -0.1306, -0.0477, -0.0855,  0.2427,
          0.1835, -0.0949],
        [ 0.0281,  0.1121, -0.1206,  0.2302, -0.0830, -0.0995, -0.0468,  0.1616,
          0.0338, -0.0433]], grad_fn=<AddmmBackward0>)

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
net(X)

tensor(-0.0826, grad_fn=<SumBackward0>)

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0062, grad_fn=<SumBackward0>)